# Homework 5: Training a classifier

## Introduction

In this homework, you will have the opportunity to train your best classifier on a data set provided on Kaggle for a current competition: 
https://www.kaggle.com/c/porto-seguro-safe-driver-prediction 

The data has slightly been transformed from its original form and down-sampled but you can find a description of the data here:
https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/data

You can also find inspiration on how to improve your model here:
https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/kernels

The "id" column is a unique identifier and has to be used as an index. The columns you want to predict is the column "target". All the other columns can be used in training.

## The files

You will 3 files with this homework:
- train_data.csv
- test_data.csv
- submission_example.csv

The first file will be used for training and contains all the attributes, the target and a unique identifier. The second file will be used to assess your classifier and contains only the attributes along with a unique identifier. The last file is an example of a submission.

## Machine Learning Libraries 

There are many Machine Learning (ML) Libraries available for python. The most known one is obviously [scikit-learn](http://scikit-learn.org/stable/supervised_learning.html) where many classifier algorithms are available for you to try. Currently [XGBoost](http://xgboost.readthedocs.io/en/latest/python/python_intro.html), [LightGBM](https://github.com/Microsoft/LightGBM) and [H2O](http://h2o-release.s3.amazonaws.com/h2o/rel-weierstrass/7/index.html) are amount the best performing ML libraries for supervised learning tasks. You are invited and advised to try all packages to find what works best for the data in this homework. 

Very popular, are also all the Artificial Neural Network libraries that can be more difficult to tame:
- [TensorFlow](https://www.tensorflow.org/)
- [Pytorch](http://pytorch.org/)
- [Keras](https://keras.io/)
- [MXNet](http://mxnet.incubator.apache.org/)
- ...

## Training a Classifier

When it comes training a classifier there are different techniques to consider to improving performance:
- Tuning the hyperparameters: find the right algorithm parameters. 
- Features selection: filter away the features that may be detrimental to the performance.
- Data transformation: transform the data to make it easier for a learner to learn from. 
- Features augmentation: create new features from the old ones.
- ...

It will be important for you to cross-validate your results to find the optimal parameters and features. There are many functions in scikit-learn that can help you achieve that:
- [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) are two examples of functions that can help you tune your models
- Scikit-learn has a variety of methods for [feature selection](http://scikit-learn.org/stable/modules/feature_selection.html)
- There are many ways to [preprocess a data set](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features). [Box-cox transformation](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.stats.boxcox.html) is also a classic method.
- It can be interesting to use unsupervised learning techniques to [create new features](http://scikit-learn.org/stable/unsupervised_learning.html)

Fundamental problems you will need to solve when it comes to preprocessing the data is to find a way to deal with missing values and categorical variables.

## The grading

The grading will be two parts:
- In the first part, you will explain in details what you have tried and what got you your best performance. You will describe the difficulties you encountered and how you solved them. The format of this small report does not need to be fancy. (10 points) 
- The second part, you will submit your predictions of a test set. You will be able to submit as many times as you want and only your best score will be retained. The performance will be assessed with AUC. (10 points)

To establish your grade of the second part, rules of the game are simple. This following code

```
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

train_df = pd.read_csv("./train_data.csv", index_col=["id"])
test_X = pd.read_csv("./test_data.csv", index_col=["id"])
test_Y = pd.read_csv("./test_target.csv", index_col=["id"])

col_cat = train_df.select_dtypes(include=[object]).columns
for col in col_cat:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].fillna("-1"))
    test_X[col] = le.transform(test_X[col].fillna("-1"))
    
train_df.fillna(-1, inplace=True)
test_X.fillna(-1, inplace=True)

clf = LogisticRegression(n_jobs=-1, C=1e6)
y_train = train_df["target"]
X_train =  train_df.drop("target", 1)
clf.fit(X_train, y_train)

metrics.roc_auc_score(test_Y, clf.predict_proba(test_X)[:, 1])
```

yields a AUC = 0.619522. If you get this AUC or lower you get 0 points. After a quick but more in-depth train, I was able to obtain AUC = 0.64486707. If you get this AUC or higher, you get the full points. Every intermediary performance will give you an intermediary grade according to the following scheme:

In [16]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
bins = np.linspace(0.619522, 0.64486707, 10)
mean_bins = (bins[1:] +  bins[:-1]) / 2
pd.DataFrame({"Intervals": pd.cut(mean_bins, bins, precision=6), "Grade": rankdata(mean_bins)})

,Grade,Intervals
0,1.0,"(0.619522, 0.622338]"
1,2.0,"(0.622338, 0.625154]"
2,3.0,"(0.625154, 0.62797]"
3,4.0,"(0.62797, 0.630786]"
4,5.0,"(0.630786, 0.633603]"
5,6.0,"(0.633603, 0.636419]"
6,7.0,"(0.636419, 0.639235]"
7,8.0,"(0.639235, 0.642051]"
8,9.0,"(0.642051, 0.644867]"


Everyday, I will publish on Piazza your best performance to see where you stand.

## Submission

You will need to submit a .csv file with exactly the following format (you can see the example in the current folder):

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

train_df = pd.read_csv("./train_data.csv", index_col=["id"])
test_X = pd.read_csv("./test_data.csv", index_col=["id"])

col_cat = train_df.select_dtypes(include=[object]).columns
for col in col_cat:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].fillna("-1"))
    test_X[col] = le.transform(test_X[col].fillna("-1"))

train_df.fillna(-1, inplace=True)
test_X.fillna(-1, inplace=True)

clf = LogisticRegression(n_jobs=-1, C=1e6)
y_train = train_df["target"]
X_train =  train_df.drop("target", 1)
clf.fit(X_train, y_train)

submission_example = pd.DataFrame({"prediction": clf.predict_proba(test_X)[:, 1]}, index=test_X.index) 
submission_example.to_csv("./submission_example.csv")